In [2]:
# Make division default to floating-point, saving confusion
from __future__ import division
from __future__ import print_function

# Allowed libraries
import numpy as np
import pandas as pd
import scipy as sp
import scipy.special
import heapq as pq
import matplotlib as mp
import matplotlib.pyplot as plt
import math
from itertools import product, combinations
from collections import OrderedDict as odict
import collections
from graphviz import Digraph, Graph
from tabulate import tabulate
import copy
import sys
import os
import datetime

In [3]:
#Headings for Location
locations = ["r1","r2","r3","r4","r5","r6","r7","r8","r9","r10","c1","c2","c3","outside"]

#Loading data
with open('data1.csv') as file:
    data1 = pd.read_csv(file)

with open('data2.csv') as file:
    data2 = pd.read_csv(file)

#Get data with relevant information
data_1 = data1[locations]

print(data_1)

data_2 = data2[locations]

      r1  r2  r3  r4  r5  r6  r7  r8  r9  r10  c1  c2  c3  outside
0      0   0  18   0   0   0   0   0   0    0   2   0   0        0
1      0   0   0   2   0   0   0   0   0    0  14   4   0        0
2      0   0   0   7   1   0   0   0   0    4   1   4   3        0
3      0   1   0   4   1   1   1   0   0    4   2   1   5        0
4      1   2   0   4   1   1   3   0   0    4   2   0   2        0
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..      ...
2396   0   0   2   0   0   0   0   0   0    0   0   0   0       18
2397   0   0   2   0   0   0   0   0   0    0   0   0   0       18
2398   0   0   2   0   0   0   0   0   0    0   0   0   0       18
2399   0   0   2   0   0   0   0   0   0    0   0   0   0       18
2400   0   0   2   0   0   0   0   0   0    0   0   0   0       18

[2401 rows x 14 columns]


In [4]:
#Loop through both data set to get potential outcomeSpace

#Initialise Empty Dict
potential_OutcomeSpace = {"r1" : set(),
                          "r2" : set(),
                          "r3" : set(),
                          "r4" : set(),
                          "r5" : set(),
                          "r6" : set(),
                          "r7" : set(),
                          "r8" : set(),
                          "r9" : set(),
                          "r10" : set(),
                          "c1" : set(),
                          "c2" : set(),
                          "c3" : set(),
                          "outside" : set()}

#Get previous none empty column
previous_nonEmpty = []

#Loop Through data 1
for index in range(len(data_1)):
    #Add to potential_OutcomeSpace if current area 
    if index != 0:
        for col in data_1.columns.tolist():
            if data_1[col][index] > data_1[col][index - 1]:
                for val in previous_nonEmpty:
                    potential_OutcomeSpace[val].add(col)
    
    #Update previous_nonEmpty
    previous_nonEmpty = []
    for col in data_1.columns.tolist():
        if data_1[col][index] > 0:
            previous_nonEmpty.append(col)

#Loop Through data 2
for index in range(len(data_2)):
    #Add to potential_OutcomeSpace if current area 
    if index != 0:
        for col in data_2.columns.tolist():
            if data_2[col][index] > data_2[col][index - 1]:
                 for val in previous_nonEmpty:
                    potential_OutcomeSpace[val].add(col)
    
    #Update previous_nonEmpty
    previous_nonEmpty = []
    for col in data_2.columns.tolist():
        if data_2[col][index] > 0:
            previous_nonEmpty.append(col)

print(potential_OutcomeSpace)


{'r1': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r8', 'c2', 'r6', 'r9', 'c3', 'r2', 'r3'}, 'r2': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r8', 'c2', 'r9', 'r6', 'c3', 'r2', 'r3'}, 'r3': {'r1', 'r10', 'outside', 'r5', 'r2', 'c1', 'r4', 'r7', 'r6', 'r8', 'r9', 'c3', 'c2', 'r3'}, 'r4': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r6', 'r8', 'c2', 'r9', 'c3', 'r2', 'r3'}, 'r5': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r6', 'r8', 'c2', 'r9', 'c3', 'r2', 'r3'}, 'r6': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r8', 'c2', 'r9', 'r6', 'c3', 'r2', 'r3'}, 'r7': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r8', 'c2', 'r9', 'r6', 'c3', 'r2', 'r3'}, 'r8': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r8', 'c2', 'r9', 'r6', 'c3', 'r2', 'r3'}, 'r9': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r8', 'c2', 'r9', 'r6', 'c3', 'r2', 'r3'}, 'r10': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r6', 'r8', 'c2', 'r9', 'c3', 'r2', 'r3'}, 'c1': {'r1', 'r10'

In [5]:
#This is not suitable as it is too dense lets try another method

# {'r1': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'r2': {'r10', 'c3', 'r9', 'c2', 'r2', 'r6', 'r4', 'r5', 'outside', 'r1', 'r7', 'r8', 'r3', 'c1'}, 
#  'r3': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'r4': {'r10', 'c3', 'r9', 'c2', 'r2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'r5': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r4', 'r5', 'outside', 'r1', 'r7', 'r8', 'r3', 'c1'}, 
#  'r6': {'r10', 'c3', 'r9', 'c2', 'r2', 'r6', 'r4', 'r5', 'outside', 'r1', 'r7', 'r8', 'r3', 'c1'}, 
#  'r7': {'r10', 'c3', 'r9', 'c2', 'r2', 'r6', 'r4', 'r5', 'outside', 'r1', 'r7', 'r8', 'r3', 'c1'}, 
#  'r8': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'r9': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'r10': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r4', 'r5', 'outside', 'r1', 'r7', 'r8', 'r3', 'c1'}, 
#  'c1': {'r10', 'c3', 'r9', 'c2', 'r2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'c2': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'c3': {'r10', 'c3', 'r9', 'c2', 'r2', 'r6', 'r4', 'r5', 'outside', 'r1', 'r7', 'r8', 'r3', 'c1'}, 
#  'outside': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}}

In [6]:
#This time attempt to only add areas that decrease
#Initialise Empty Dict
potential_OutcomeSpace = {"r1" : set(),
                          "r2" : set(),
                          "r3" : set(),
                          "r4" : set(),
                          "r5" : set(),
                          "r6" : set(),
                          "r7" : set(),
                          "r8" : set(),
                          "r9" : set(),
                          "r10" : set(),
                          "c1" : set(),
                          "c2" : set(),
                          "c3" : set(),
                          "outside" : set()}

#Get previous none empty column
previous_nonEmpty = []

#Loop Through data 1
for index in range(len(data_1)):
    #Add to potential_OutcomeSpace if current area 
    if index != 0:
        #Update previous_nonEmpty
        previous_nonEmpty = []
        for col in data_1.columns.tolist():
            if data_1[col][index] < data_1[col][index - 1]:
                previous_nonEmpty.append(col)

        for col in data_1.columns.tolist():
            if data_1[col][index] > data_1[col][index - 1]:
                for val in previous_nonEmpty:
                    potential_OutcomeSpace[val].add(col)
    

#Loop Through data 2
for index in range(len(data_2)):
    #Add to potential_OutcomeSpace if current area 
    if index != 0:
        #Update previous_nonEmpty
        previous_nonEmpty = []
        for col in data_2.columns.tolist():
            if data_2[col][index] < data_2[col][index - 1]:
                previous_nonEmpty.append(col)

        for col in data_2.columns.tolist():
            if data_2[col][index] > data_2[col][index - 1]:
                for val in previous_nonEmpty:
                    potential_OutcomeSpace[val].add(col)
    
print(potential_OutcomeSpace)

{'r1': {'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r6', 'c2', 'r9', 'r8', 'c3', 'r2', 'r3'}, 'r2': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r8', 'c2', 'r9', 'r6', 'c3', 'r3'}, 'r3': {'r1', 'r10', 'outside', 'r5', 'r2', 'c1', 'r4', 'r7', 'r8', 'r6', 'r9', 'c3', 'c2'}, 'r4': {'r1', 'r10', 'outside', 'r5', 'c1', 'r7', 'r6', 'r8', 'c2', 'r9', 'c3', 'r2', 'r3'}, 'r5': {'r1', 'r10', 'outside', 'c1', 'r4', 'r7', 'r6', 'c2', 'r8', 'r9', 'c3', 'r2', 'r3'}, 'r6': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r7', 'r8', 'c2', 'r9', 'c3', 'r2', 'r3'}, 'r7': {'r1', 'r10', 'outside', 'r5', 'c1', 'r4', 'r8', 'c2', 'r9', 'r6', 'c3', 'r2', 'r3'}, 'r8': {'r1', 'r10', 'outside', 'r5', 'r2', 'c1', 'r4', 'r7', 'r6', 'c2', 'r9', 'c3', 'r3'}, 'r9': {'r1', 'r10', 'outside', 'r5', 'r2', 'c1', 'r4', 'r7', 'r8', 'c2', 'r6', 'c3', 'r3'}, 'r10': {'r1', 'outside', 'r5', 'c1', 'r4', 'r7', 'r8', 'c2', 'r6', 'r9', 'c3', 'r2', 'r3'}, 'c1': {'r1', 'r10', 'outside', 'r5', 'r2', 'r4', 'r7', 'r6', 'c2', 'r8', 'r9', 

In [7]:
#This is not suitable as it is too dense lets try another method

# {'r1': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r5', 'outside', 'c1', 'r7', 'r8', 'r3', 'r4'}, 
#  'r2': {'r10', 'c3', 'r9', 'c2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'r3': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r4'}, 
#  'r4': {'r9', 'c3', 'r10', 'r2', 'c2', 'r6', 'r5', 'outside', 'r1', 'r7', 'r8', 'r3', 'c1'}, 
#  'r5': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'r6': {'r10', 'c3', 'r9', 'c2', 'r2', 'r4', 'r5', 'outside', 'r1', 'r7', 'r8', 'r3', 'c1'}, 
#  'r7': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r5', 'c1', 'r1', 'outside', 'r8', 'r3', 'r4'}, 
#  'r8': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'outside', 'r5', 'c1', 'r1', 'r7', 'r3', 'r4'}, 
#  'r9': {'r10', 'c3', 'c2', 'r2', 'r6', 'outside', 'r5', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'r10': {'r9', 'c3', 'r2', 'c2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'c1': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r5', 'outside', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'c2': {'r10', 'c3', 'r9', 'r2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'c3': {'r10', 'r9', 'c2', 'r2', 'r6', 'r5', 'outside', 'c1', 'r1', 'r7', 'r8', 'r3', 'r4'}, 
#  'outside': {'r10', 'c3', 'r9', 'r2', 'c2', 'r6', 'r4', 'r5', 'r1', 'r7', 'r8', 'r3', 'c1'}}

In [8]:
#Essentially Counting the number of people decrease in that room (from T-1 to T) and then count the number of increase in other rooms (from T-1 to T)
#If a room increases, count the possible number of transfers to that room from current room so either take number of increase or number of decrease depending which one is smaller
#Squared value for weights

# Initialize state transition DataFrame
state_transitions = pd.DataFrame(0, index=data_1.columns, columns=data_1.columns)

# Current empty
nonEmpty = {"r1": 0, "r2": 0, "r3": 0, "r4": 0, "r5": 0, "r6": 0, "r7": 0, "r8": 0, "r9": 0, "r10": 0, "c1": 0, "c2": 0, "c3": 0, "outside": 0}

# Loop Through data_1
for index in range(1, len(data_1)):
    # reset nonEmpty
    for key in nonEmpty:
        nonEmpty[key] = 0

    # Get values that increased
    for key in nonEmpty:
        if data_1[key][index] > data_1[key][index - 1]:
            nonEmpty[key] = data_1[key][index] - data_1[key][index - 1]

    # Loop through each column
    for col in data_1.columns:
        if data_1[col][index] < data_1[col][index - 1]:
            num = data_1[col][index - 1] - data_1[col][index]
            # If not the same area, add either the count of the increase or the decrease
            for key in nonEmpty:
                if key != col:
                    if nonEmpty[key] < num:
                        state_transitions.at[col, key] += nonEmpty[key]**2
                    else:
                        state_transitions.at[col, key] += num**2
                else:
                    state_transitions.at[key, key] += data_1[key][index]**2

# Loop Through data_2
for index in range(1, len(data_2)):
    # reset nonEmpty
    for key in nonEmpty:
        nonEmpty[key] = 0

    # Get values that increased
    for key in nonEmpty:
        if data_2[key][index] > data_2[key][index - 1]:
            nonEmpty[key] = data_2[key][index] - data_2[key][index - 1]

    # Loop through each column
    for col in data_2.columns:
        if data_2[col][index] < data_2[col][index - 1]:
            num = data_2[col][index - 1] - data_2[col][index]
            # If not the same area, add either the count of the increase or the decrease
            for key in nonEmpty:
                if key != col:
                    if nonEmpty[key] < num:
                        state_transitions.at[col, key] += nonEmpty[key]**2
                    else:
                        state_transitions.at[col, key] += num**2
                else:
                    state_transitions.at[key, key] += data_2[key][index]**2

# Normalize
normalized_matrix = state_transitions / state_transitions.sum(axis=0)
normalized_matrix.to_csv('normalized_matrix.csv')

In [9]:
extended_data = pd.concat([data1, data2], ignore_index=True)
sorted_data = extended_data.sort_values(by='time')

sorted_data.to_csv('sorted_data.csv',index=False)

In [10]:
outcomeSpace = {'r1':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r2':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r3':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r4':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r5':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r6':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r7':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r8':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r9':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r10':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'c1':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'c2':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'c3':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'outside':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside']
}

def count_tran(data,tar):
    d1 = list(data[tar])
    d2 = d1[1:]
    d2.append(0)
    d1 = np.array(d1)
    if d1.sum() == 0:
        return 0
    d2 = np.array(d2)
    d = d1-d2
    sum = 0
    for once in d:
        if once > 0:
            sum += once

    return sum/d1.sum()

#index is the dictionary, key is the space and value is the corresponding index number.
index = {}
start = 0
for key in outcomeSpace.keys():
    index[key] = start
    start+=1


data = pd.read_csv('sorted_data.csv')

#output is the 5 time period transaction matrix, will be stored in tran_matrix.csv file.
output = {}

#first time period (8:00 - 8:05)
for key in outcomeSpace.keys():
    #the transaction rate of the specific space. Inialize it to be all 0.
    tran = [0]*14

    #count the possiblity of moving to other rooms
    for i in range(len(data[:38])-1):
        t1_d = data.iloc[i]
        t2_d = data.iloc[i+1]

        if int(t2_d[key]) < int(t1_d[key]):
            diff = int(t1_d[key]) - int(t2_d[key])
            for mov in outcomeSpace[key]:
                t1_dm = t1_d[mov]
                t2_dm = t2_d[mov]
                diff_m = int(t2_dm)-int(t1_dm)
                if diff_m >= diff:
                    tran[index[mov]] += diff
                    break
                else:
                    if diff_m >0:
                        tran[index[mov]] += diff_m
                        diff -= diff_m
    
    
    d1 = list(data[:38][key])
    d1 = np.array(d1)
    if d1.sum() != 0:
        tran[:] = [x/d1.sum() for x in tran]
    tran[index[key]] = 1 - count_tran(data[:38], key)
    output[key+'_s1'] = tran

#second time period (8:05 - 17:30)
for key in outcomeSpace.keys():
    tran = [0]*14
    #count the possiblity of moving to other rooms
    for i in range(len(data[38:4558])-1):
        t1_d = data.iloc[38+i]
        t2_d = data.iloc[38+i+1]

        if int(t2_d[key]) < int(t1_d[key]):
            diff = int(t1_d[key]) - int(t2_d[key])
            for mov in outcomeSpace[key]:
                t1_dm = t1_d[mov]
                t2_dm = t2_d[mov]
                diff_m = int(t2_dm)-int(t1_dm)
                if diff_m >= diff:
                    tran[index[mov]] += diff
                    break
                else:
                    if diff_m >0:
                        tran[index[mov]] += diff_m
                        diff -= diff_m
    d1 = list(data[38:4558][key])
    d1 = np.array(d1)
    if d1.sum() != 0:
        tran[:] = [x/d1.sum() for x in tran]
    tran[index[key]] = 1 - count_tran(data[38:4558], key)
    output[key+'_s2'] = tran

#third time period (17:30 - 18:00)
for key in outcomeSpace.keys():
    tran = [0]*14

    #count the possiblity of moving to other rooms
    for i in range(len(data[4558:])-1):
        t1_d = data.iloc[4558+i]
        t2_d = data.iloc[4558+i+1]

        if int(t2_d[key]) < int(t1_d[key]):
            diff = int(t1_d[key]) - int(t2_d[key])
            for mov in outcomeSpace[key]:
                t1_dm = t1_d[mov]
                t2_dm = t2_d[mov]
                diff_m = int(t2_dm)-int(t1_dm)
                if diff_m >= diff:
                    tran[index[mov]] += diff
                    break
                else:
                    if diff_m >0:
                        tran[index[mov]] += diff_m
                        diff -= diff_m
    d1 = list(data[4558:][key])
    d1 = np.array(d1)
    if d1.sum() != 0:
        tran[:] = [x/d1.sum() for x in tran]
    if count_tran(data[4558:], key) == 0:
        tran[index[key]] = 0
    else:
        tran[index[key]] = 1 - count_tran(data[4558:], key)
    output[key+'_s3'] = tran

#output to csv file.
state_csv = pd.DataFrame(output)
state_csv.to_csv('state_matrix.csv',index=False)

In [11]:
outcomeSpace = {'r1':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r2':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r3':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r4':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r5':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r6':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r7':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r8':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r9':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'r10':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'c1':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'c2':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'c3':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside'],
                'outside':['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','c1','c2','c3','outside']
}

def count_tran(data,tar):
    d1 = list(data[tar])
    d2 = d1[1:]
    d2.append(0)
    d1 = np.array(d1)
    if d1.sum() == 0:
        return 0
    d2 = np.array(d2)
    d = d1-d2
    sum = 0
    for once in d:
        if once > 0:
            sum += once

    return sum/d1.sum()

#index is the dictionary, key is the space and value is the corresponding index number.
index = {}
start = 0
for key in outcomeSpace.keys():
    index[key] = start
    start+=1


data = pd.read_csv('sorted_data.csv')

#output is the 5 time period transaction matrix, will be stored in tran_matrix.csv file.
output = {}

#first time period (8:00 - 8:05)
for key in outcomeSpace.keys():
    #the transaction rate of the specific space. Inialize it to be all 0.
    tran = [0]*14

    #count the possiblity of moving to other rooms
    for i in range(len(data[:38])-1):
        t1_d = data.iloc[i]
        t2_d = data.iloc[i+1]

        if int(t2_d[key]) < int(t1_d[key]):
            diff = int(t1_d[key]) - int(t2_d[key])
            for mov in outcomeSpace[key]:
                t1_dm = t1_d[mov]
                t2_dm = t2_d[mov]
                diff_m = int(t2_dm)-int(t1_dm)
                if diff_m >= diff:
                    tran[index[mov]] += diff
                    break
                else:
                    if diff_m >0:
                        tran[index[mov]] += diff_m
                        diff -= diff_m
    
    
    d1 = list(data[:38][key])
    d1 = np.array(d1)
    if d1.sum() != 0:
        tran[:] = [x/d1.sum() for x in tran]
    tran[index[key]] = 1 - count_tran(data[:38], key)
    output[key+'_s1'] = tran

#second time period (8:05 - 12:00)
for key in outcomeSpace.keys():
    tran = [0]*14
    #count the possiblity of moving to other rooms
    for i in range(len(data[38:1918])-1):
        t1_d = data.iloc[38+i]
        t2_d = data.iloc[38+i+1]

        if int(t2_d[key]) < int(t1_d[key]):
            diff = int(t1_d[key]) - int(t2_d[key])
            for mov in outcomeSpace[key]:
                t1_dm = t1_d[mov]
                t2_dm = t2_d[mov]
                diff_m = int(t2_dm)-int(t1_dm)
                if diff_m >= diff:
                    tran[index[mov]] += diff
                    break
                else:
                    if diff_m >0:
                        tran[index[mov]] += diff_m
                        diff -= diff_m
    d1 = list(data[38:1918][key])
    d1 = np.array(d1)
    if d1.sum() != 0:
        tran[:] = [x/d1.sum() for x in tran]
    tran[index[key]] = 1 - count_tran(data[38:1918], key)
    output[key+'_s2'] = tran

#third time period (12:00 - 14:00)
for key in outcomeSpace.keys():
    tran = [0]*14
    #count the possiblity of moving to other rooms
    for i in range(len(data[1918:2878])-1):
        t1_d = data.iloc[2878+i]
        t2_d = data.iloc[2878+i+1]

        if int(t2_d[key]) < int(t1_d[key]):
            diff = int(t1_d[key]) - int(t2_d[key])
            for mov in outcomeSpace[key]:
                t1_dm = t1_d[mov]
                t2_dm = t2_d[mov]
                diff_m = int(t2_dm)-int(t1_dm)
                if diff_m >= diff:
                    tran[index[mov]] += diff
                    break
                else:
                    if diff_m >0:
                        tran[index[mov]] += diff_m
                        diff -= diff_m
    d1 = list(data[1918:2878][key])
    d1 = np.array(d1)
    if d1.sum() != 0:
        tran[:] = [x/d1.sum() for x in tran]
    tran[index[key]] = 1 - count_tran(data[1918:2878], key)
    output[key+'_s3'] = tran

#fourth time period (14:00 - 17:30)
for key in outcomeSpace.keys():
    tran = [0]*14
    #count the possiblity of moving to other rooms
    for i in range(len(data[2878:4558])-1):
        t1_d = data.iloc[2878+i]
        t2_d = data.iloc[2878+i+1]

        if int(t2_d[key]) < int(t1_d[key]):
            diff = int(t1_d[key]) - int(t2_d[key])
            for mov in outcomeSpace[key]:
                t1_dm = t1_d[mov]
                t2_dm = t2_d[mov]
                diff_m = int(t2_dm)-int(t1_dm)
                if diff_m >= diff:
                    tran[index[mov]] += diff
                    break
                else:
                    if diff_m >0:
                        tran[index[mov]] += diff_m
                        diff -= diff_m
    d1 = list(data[2878:4558][key])
    d1 = np.array(d1)
    if d1.sum() != 0:
        tran[:] = [x/d1.sum() for x in tran]
    tran[index[key]] = 1 - count_tran(data[2878:4558], key)
    output[key+'_s4'] = tran


#fifth time period (17:30 - 18:00)
for key in outcomeSpace.keys():
    tran = [0]*14

    #count the possiblity of moving to other rooms
    for i in range(len(data[4558:])-1):
        t1_d = data.iloc[4558+i]
        t2_d = data.iloc[4558+i+1]

        if int(t2_d[key]) < int(t1_d[key]):
            diff = int(t1_d[key]) - int(t2_d[key])
            for mov in outcomeSpace[key]:
                t1_dm = t1_d[mov]
                t2_dm = t2_d[mov]
                diff_m = int(t2_dm)-int(t1_dm)
                if diff_m >= diff:
                    tran[index[mov]] += diff
                    break
                else:
                    if diff_m >0:
                        tran[index[mov]] += diff_m
                        diff -= diff_m
    d1 = list(data[4558:][key])
    d1 = np.array(d1)
    if d1.sum() != 0:
        tran[:] = [x/d1.sum() for x in tran]
    if count_tran(data[4558:], key) == 0:
        tran[index[key]] = 0
    else:
        tran[index[key]] = 1 - count_tran(data[4558:], key)
    output[key+'_s5'] = tran

#output to csv file.
state_csv = pd.DataFrame(output)
state_csv.to_csv('state_matrix_5.csv',index=False)

In [12]:
outcomeSpace = {
    'r1': ['r2'], 
    'r2': ['r1', 'c3'], 
    'r3': ['outside', 'c1', 'c2'], 
    'r4': ['r5', 'c1', 'c2'], 
    'r5': ['r6', 'r4'], 
    'r6': ['r5'], 
    'r7': ['c3', 'r8'], 
    'r8': ['r7'], 
    'r9': ['c1', 'c2'], 
    'r10': ['c3', 'c1', 'c2'], 
    'c1': ['r3', 'r4', 'r9'], 
    'c2': ['r3', 'r4', 'r9', 'r10'], 
    'c3': ['r2', 'r7', 'r10'], 
    'outside': ['r3']
}

def count_tran(data,tar):
    d1 = list(data[tar])
    d2 = d1[1:]
    d2.append(0)
    d1 = np.array(d1)
    if d1.sum() == 0:
        return 0
    d2 = np.array(d2)
    d = d1-d2
    sum = 0
    for once in d:
        if once > 0:
            sum += once

    return sum/d1.sum()

#index is the dictionary, key is the space and value is the corresponding index number.
index = {}
start = 0
for key in outcomeSpace.keys():
    index[key] = start
    start+=1


data = pd.read_csv('sorted_data.csv')

#output is the 5 time period transaction matrix, will be stored in tran_matrix.csv file.
output = {}

#first time period (8:00 - 8:05)
for key in outcomeSpace.keys():
    #the transaction rate of the specific space. Inialize it to be all 0.
    tran = [0]*14

    #count the possiblity of moving to other rooms
    for i in range(len(data[:38])-1):
        t1_d = data.iloc[i]
        t2_d = data.iloc[i+1]

        if int(t2_d[key]) < int(t1_d[key]):
            diff = int(t1_d[key]) - int(t2_d[key])
            for mov in outcomeSpace[key]:
                t1_dm = t1_d[mov]
                t2_dm = t2_d[mov]
                diff_m = int(t2_dm)-int(t1_dm)
                if diff_m >= diff:
                    tran[index[mov]] += diff
                    break
                else:
                    if diff_m >0:
                        tran[index[mov]] += diff_m
                        diff -= diff_m
    
    
    d1 = list(data[:38][key])
    d1 = np.array(d1)
    if d1.sum() != 0:
        tran[:] = [x/d1.sum() for x in tran]
    tran[index[key]] = 1 - count_tran(data[:38], key)
    output[key+'_s1'] = tran

#second time period (8:05 - 17:30)
for key in outcomeSpace.keys():
    tran = [0]*14
    #count the possiblity of moving to other rooms
    for i in range(len(data[38:4558])-1):
        t1_d = data.iloc[38+i]
        t2_d = data.iloc[38+i+1]

        if int(t2_d[key]) < int(t1_d[key]):
            diff = int(t1_d[key]) - int(t2_d[key])
            for mov in outcomeSpace[key]:
                t1_dm = t1_d[mov]
                t2_dm = t2_d[mov]
                diff_m = int(t2_dm)-int(t1_dm)
                if diff_m >= diff:
                    tran[index[mov]] += diff
                    break
                else:
                    if diff_m >0:
                        tran[index[mov]] += diff_m
                        diff -= diff_m
    d1 = list(data[38:4558][key])
    d1 = np.array(d1)
    if d1.sum() != 0:
        tran[:] = [x/d1.sum() for x in tran]
    tran[index[key]] = 1 - count_tran(data[38:4558], key)
    output[key+'_s2'] = tran

#third time period (17:30 - 18:00)
for key in outcomeSpace.keys():
    tran = [0]*14

    #count the possiblity of moving to other rooms
    for i in range(len(data[4558:])-1):
        t1_d = data.iloc[4558+i]
        t2_d = data.iloc[4558+i+1]

        if int(t2_d[key]) < int(t1_d[key]):
            diff = int(t1_d[key]) - int(t2_d[key])
            for mov in outcomeSpace[key]:
                t1_dm = t1_d[mov]
                t2_dm = t2_d[mov]
                diff_m = int(t2_dm)-int(t1_dm)
                if diff_m >= diff:
                    tran[index[mov]] += diff
                    break
                else:
                    if diff_m >0:
                        tran[index[mov]] += diff_m
                        diff -= diff_m
    d1 = list(data[4558:][key])
    d1 = np.array(d1)
    if d1.sum() != 0:
        tran[:] = [x/d1.sum() for x in tran]
    if count_tran(data[4558:], key) == 0:
        tran[index[key]] = 0
    else:
        tran[index[key]] = 1 - count_tran(data[4558:], key)
    output[key+'_s3'] = tran

#output to csv file.
state_csv = pd.DataFrame(output)
state_csv.to_csv('state_matrix_6.csv',index=False)